# AI Lab Assignment 1

- Course: **SEZC444**
- Student ID: **2020mt93555**
- Student name : **Paul Cleetus**



## Problem Statement

**A** Implement the Learning Agent which is in search of the only gold box in the given static environment equipped with the metal detectors . The metal detectors are capable of sensing the metal in their own and all surrounding cells in their circumference.

While the agent initial battery is at 10000 points. Each movement of the agent reduces the battery by 1 point and at a time the agent can move only one cell either of the forward, backward, left or right directions. If the agent moves in to metal detector sensing range, the agent’s battery life will reduce by 100 battery points. The ultimate goal is to catch the gold box and it results 10000 additional battery points. The agent can sense only its battery points, which are automatically updated on each movement and the resultant cell contents. 
```
7 marks 
1 mark for mapping to correct concept (among our topics), 
1 mark for briefing the learning parameters and 
5 marks for an error-free executing model
```



## PEAS Description

**Performance Measure**
- Each Movement : -1 point 
- Adjescant to Metal Detector : -100 points
- Gold Box found : +10000 points 

**Environment**
- A 10 x 10 grid of rooms denoted Rows 1 to 10, Columns A to J
- Initial Battery : 10000 points 

**Actuators**
- Agent can move 1 step Forward / Backward / Left / Right
- If the movement is againt a wall then there is no effect 

**Sensor**
The agent can sense only its battery points, which are automatically updated on each movement and the resultant cell contents.


## Learning Approach

I am using the **Q-Learning** Re-inforcement learning technique to solve this problem .



In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import string
from pprint import pprint

# Define the Row and Column range for the environmnet
rows    = np.arange(1,11)
columns = list(string.ascii_uppercase[0:10])

# Specify the colours for various elements in environment
gold     = 4.7
detector = 3.9
magfield = 2.2
agent    = 5.5
empty    = 0.0
temp     = 0.0

# Specify the text annotations to be plotted in the output for each element
notation = {
    gold     : 'G',
    detector : 'D',
    magfield : 'X',
    agent    : 'A',
    empty    : '',
}

# Specify the performance measures
battery        = 10000
reward   = 0
feedback = {
    gold     : 10000,
    detector :  -100,
    magfield :  -100,
    agent    :     0,
    empty    :    -1,
    'wall'   :  -100,
}

# define actions
#numeric action codes: 0 = up, 1 = right, 2 = down, 3 = left
actions = ['up', 'right', 'down', 'left']

# Some Global Variables for the environment
agent_location = []
next_location = []
agent_stop = False
environment = np.array([])


#define training parameters
epsilon = 0.95 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.95 #discount factor for future rewards
learning_rate = 0.95 #the rate at which the AI agent should learn

q_values = np.zeros((10, 10, 4)) #the q-table for 10x10 matrix and 4 actions 

In [ ]:
# intialize the Matrix environment as given in the question
def createEnvironment():
    global agent_location
    global next_location
    global agent_stop
    global environment
    
    # agent location
    agent_location = [9,0]
    next_location  = [9,0]
    #next_location  = agent_location.copy()

    # agent can stop
    agent_stop  = False

    # Specify the environmnet as given in the Problem Statement for the assignment
    environment = np.array(
                    [[empty, empty, empty, empty, empty, empty, empty, magfield, magfield, magfield],    # 1
                     [empty, empty, magfield, magfield, magfield, empty, empty, magfield, detector, magfield],    # 2
                     [empty, empty, magfield, detector, magfield, empty, magfield, magfield, magfield, magfield],    # 3
                     [magfield, magfield, magfield, magfield, magfield, empty, magfield, detector, magfield, empty],    # 4
                     [magfield, detector, magfield, magfield, magfield, empty, magfield, magfield, magfield, empty],    # 5
                     [magfield, magfield,magfield, detector, magfield, empty, empty, empty, empty, empty],    # 6
                     [empty, empty, magfield, magfield, magfield, empty, empty, empty, empty, empty],    # 7
                     [empty, empty, empty, empty, empty, empty, magfield, magfield, magfield, empty],    # 8
                     [empty, magfield, magfield, magfield, empty, empty, magfield, detector, magfield, empty],    # 9
                     [agent, magfield, detector, magfield, empty, empty, magfield, magfield, magfield, gold]] #10
                )



In [ ]:
# Display the matrix plot
def displayPlot(environment):
    fig, ax = plt.subplots()
    im = ax.imshow(environment)

    # Show all rooms
    ax.set_xticks(np.arange(len(columns)))
    ax.set_yticks(np.arange(len(rows)))

    # Label with the respective list entries
    ax.set_xticklabels(columns)
    ax.set_yticklabels(rows)


    # Loop over data dimensions and create text annotations.
    for i in range(len(rows)):
        for j in range(len(columns)):
            text = ax.text(j, i, notation.get(environment[i, j]),
                       ha="center", va="center", color="w")

    ax.set_title("Q-Learning Agent searching the Gold box among metal detector array")
    fig.tight_layout()
    plt.show()



In [ ]:
# Move the agent if not hitting the wall
def moveAgent():

#    if moveDirection == "left" and next_location[1] > 0:
#        next_location[1] = next_location[1] - 1
#    elif moveDirection == "right" and next_location[1] < len(columns) - 1:
#        next_location[1] = next_location[1] +  1
#    elif moveDirection == "forward" and next_location[0] > 0:
#        next_location[0] = next_location[0] - 1
#    elif moveDirection == "backward" and next_location[0] < len(rows) - 1:
#        next_location[0] = next_location[0] + 1
        
    global temp
    global agent_location
    global next_location
    
    updateBattery()
    
    # swap colours
    environment[agent_location[0]][agent_location[1]] = temp
    temp = environment[next_location[0]][next_location[1]]
    environment[next_location[0]][next_location[1]] = agent
    agent_location = next_location.copy()
    
    displayPlot(environment)
    
    
#define an epsilon greedy algorithm that will choose which action to take next (i.e., where to move next)
def get_next_action(current_row_index, current_column_index, epsilon):
    #if a randomly chosen value between 0 and 1 is less than epsilon, 
    #then choose the most promising value from the Q-table for this state.
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index,current_column_index])
    else: #choose a random action
        return np.random.randint(4)

#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < len(columns) - 1:
        new_column_index += 1
    elif actions[action_index] == 'down' and current_row_index <  len(rows) - 1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1  
    return new_row_index, new_column_index

In [ ]:
# Update the batter points based on the next position feedback
def updateBattery():
    global battery
    global reward
    global agent_stop
    
    if next_location[0] == agent_location[0] and next_location[1] == agent_location[1]:
        print("Agent has hit a wall")
        reward = feedback['wall']
    else:
        print("Next Environment : ", notation[environment[next_location[0]][next_location[1]]])
        reward = feedback[environment[next_location[0]][next_location[1]]]
        
    print("Feedback : ", reward)
    battery = battery + reward
    
    
    if reward == 10000:
        print("Agent has found the Gold")
        agent_stop  = True
    elif battery < 1:
        print("Agent has drained the battery")
        agent_stop  = True
        
    print("Agent Battery Level : " + str(battery))
    
    

In [ ]:
# Create the intial state and status for reference
createEnvironment()

print("This cell output initial view of the learning agent environment and Q-values table")

displayPlot(environment)
pprint(q_values)

print("Agent Battery Level : " + str(battery))
print("Feedback : " + str(reward))
print("See the Next cell output to view the learning progress step snapshots")

In [ ]:
#define training parameters
global epsilon
global discount_factor
global learning_rate
global agent_stop
global q_values
global reward
global hit_wall
global battery

#run through 5 training episodes
for episode in range(5):
    #get the starting location for this episode
    row_index, column_index = 9 , 0
    hit_wall   = False
    agent_stop = False
    battery    = 10000
    print('Episode : ', episode)
    
    pprint(q_values)
    
    #continue taking actions (i.e., moving) until we reach a terminal state
    #(i.e., until we reach the item packaging area or crash into an item storage location)
    while not agent_stop:

        #choose which action to take (i.e., where to move next)
        action_index = get_next_action(row_index, column_index, epsilon)

        #perform the chosen action, and transition to the next state (i.e., move to the next location)
        old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
        row_index, column_index = get_next_location(row_index, column_index, action_index)
            
        next_location[0] = row_index
        next_location[1] = column_index
        moveAgent()
    
        #receive the reward for moving to the new state, and calculate the temporal difference
        old_q_value = q_values[old_row_index, old_column_index, action_index]
        temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

        #update the Q-value for the previous state and action pair
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')

In [ ]:
# Print Agent's Final position
global environment
global q_values

displayPlot(environment)
        
pprint(q_values)


**B** Adjust the learning parameters for five times and return the best of those five parameter sets in terms of quick convergence towards the solution 

*3 marks*


Learning parameters:

**First run parameters** the followng parameters where used
```
epsilon = 0.5 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.9 #discount factor for future rewards
learning_rate = 0.9 #the rate at which the AI agent should learn
```

- The Gold was found first time in 2nd Episode of training iteration with around 5144 battery points remaining
- The Gold was found second time in 4th Episode of training iteration with around 7350 battery points remaining
- The Gold was found third time in 5th Episode of training iteration with around 5289 battery points remaining

**Second run parameters** the followng parameters where used
```
epsilon = 0.7 
discount_factor = 0.9 
learning_rate = 0.5 
```

- The Gold was found first time in 3rd Episode of training iteration with around 4204 battery points remaining
- The Gold was found second time in 4th Episode of training iteration with around 9370 battery points remaining

**Third run parameters** the followng parameters where used
```
epsilon = 0.85 
discount_factor = 0.9 
learning_rate = 0.95 
```
- The Gold was found first time in 1st Episode of training iteration with around 1287 battery points remaining
- The Gold was found second time in 2nd Episode of training iteration with around 6757 battery points remaining
- The Gold was found third time in 3rd Episode of training iteration with around 6509 battery points remaining
- The Gold was found fourth time in 4th Episode of training iteration with around 8818 battery points remaining


**Fourth run parameters** the followng parameters where used
```
epsilon = 0.95 
discount_factor = 0.5 
learning_rate = 0.95 
```
- The Gold was found first time in 1st Episode of training iteration with around 5053 battery points remaining
- The Gold was found second time in 2nd Episode of training iteration with around 5653 battery points remaining
- The Gold was found third time in 3rd Episode of training iteration with around 8485 battery points remaining
- The Gold was found fourth time in 4th Episode of training iteration with around 5651 battery points remaining

**Fifth run parameters** the followng parameters where used
```
epsilon = 0.95 
discount_factor = 0.95 
learning_rate = 0.95 
```
- The Gold was found first time in 2nd Episode of training iteration with around 8529 battery points remaining
- The Gold was found second time in 3rd Episode of training iteration with around 6650 battery points remaining
- The Gold was found third time in 4th Episode of training iteration with around 9054 battery points remaining


**Additonal feedback than the ones suggested in question:**
I experimented with adding in a feedback for encountering a wall,
so that the q-values for such action could also be learned by the agent as a punishment / -ve feedback
This considerably reduced the learning time and avoided repeating the decision to move into the wall always



## References

- [Creating Annotated Heatmap in Matplotlib](https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py)
-[Video : Q-Learning: A Complete Example in Python, Dr. Daniel Soper](https://www.youtube.com/watch?v=iKdlKYG78j4)
-[Reference Code: Q-Learning: A Complete Example in Python, Dr. Daniel Soper](https://colab.research.google.com/drive/1E2RViy7xmor0mhqskZV14_NUj2jMpJz3#scrollTo=z43QX_t080q3)
